In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:

'''# Load and clean data
data = pd.read_csv('care.csv')

input_columns = [
    'H_COMP_7_A', 'H_COMP_7_D_SD', 'H_COMP_7_SA',
    'H_CT_MED_A', 'H_CT_MED_D_SD', 'H_CT_MED_SA',
    'H_CT_PREFER_A', 'H_CT_PREFER_D_SD', 'H_CT_PREFER_SA',
    'H_CT_UNDER_A', 'H_CT_UNDER_D_SD', 'H_CT_UNDER_SA'
]
output_column = 'H_COMP_7_STAR_RATING'

# Preprocessing
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data = data.dropna()
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data = data.dropna()

X = data[input_columns].values
y = to_categorical(data[output_column].values.astype(int) - 1)  # One-hot encoded, 0-4

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
with open('care_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape input for CNN (samples, time steps, features per step)
# Since we have 12 features, treat each as a timestep with 1 feature
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# CNN Model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(32, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')  # 5 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate
loss, accuracy = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Save model
model.save('care_cnn.keras')

# Save model in pickle (optional but not typical for Keras)
with open('care_cnn.pkl', 'wb') as f:
    pickle.dump(model, f)

print("CNN model and scaler saved successfully.")'''


'# Load and clean data\ndata = pd.read_csv(\'care.csv\')\n\ninput_columns = [\n    \'H_COMP_7_A\', \'H_COMP_7_D_SD\', \'H_COMP_7_SA\',\n    \'H_CT_MED_A\', \'H_CT_MED_D_SD\', \'H_CT_MED_SA\',\n    \'H_CT_PREFER_A\', \'H_CT_PREFER_D_SD\', \'H_CT_PREFER_SA\',\n    \'H_CT_UNDER_A\', \'H_CT_UNDER_D_SD\', \'H_CT_UNDER_SA\'\n]\noutput_column = \'H_COMP_7_STAR_RATING\'\n\n# Preprocessing\ndata = data[input_columns + [output_column]]\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata = data.dropna()\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata = data.dropna()\n\nX = data[input_columns].values\ny = to_categorical(data[output_column].values.astype(int) - 1)  # One-hot encoded, 0-4\n\n# Train-test split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Scale features\nscaler = StandardScaler()

In [3]:
# Load the dataset
'''data = pd.read_csv('clean.csv')

# Define input and output columns
input_columns = ['H_CLEAN_HSP_A_P', 'H_CLEAN_HSP_SN_P', 'H_CLEAN_HSP_U_P']
output_column = 'H_CLEAN_STAR_RATING'

# Filter and clean
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data = data.dropna()

# Ensure numeric types
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data = data.dropna()

# Extract features and target
X = data[input_columns].values
y = data[output_column].values
y = to_categorical(y - 1)  # One-hot encode from 0 to 4

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
with open('clean_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape for Conv1D: (samples, timesteps, features)
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Build the CNN model
model = Sequential([
    Conv1D(32, kernel_size=2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')  # Output: 5 star classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Save the CNN model
model.save('clean_cnn.keras')

# Save the scaler
with open('clean_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


print("CNN model and scaler saved successfully.")'''


'data = pd.read_csv(\'clean.csv\')\n\n# Define input and output columns\ninput_columns = [\'H_CLEAN_HSP_A_P\', \'H_CLEAN_HSP_SN_P\', \'H_CLEAN_HSP_U_P\']\noutput_column = \'H_CLEAN_STAR_RATING\'\n\n# Filter and clean\ndata = data[input_columns + [output_column]]\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata = data.dropna()\n\n# Ensure numeric types\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata = data.dropna()\n\n# Extract features and target\nX = data[input_columns].values\ny = data[output_column].values\ny = to_categorical(y - 1)  # One-hot encode from 0 to 4\n\n# Train/test split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Scale features\nscaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)\nX_test_scaled = scaler.transform(X_test)\n\n# Save the scal

In [4]:
'''
# Load the dataset
data = pd.read_csv('doctor_new.csv')

# Define input and output columns
input_columns = [
    'H_COMP_2_A_P', 'H_COMP_2_SN_P', 'H_COMP_2_U_P',
    'H_DOCTOR_EXPLAIN_A_P', 'H_DOCTOR_EXPLAIN_SN_P', 'H_DOCTOR_EXPLAIN_U_P',
    'H_DOCTOR_LISTEN_A_P', 'H_DOCTOR_LISTEN_SN_P', 'H_DOCTOR_LISTEN_U_P',
    'H_DOCTOR_RESPECT_A_P', 'H_DOCTOR_RESPECT_SN_P', 'H_DOCTOR_RESPECT_U_P'
]
output_column = 'H_COMP_2_STAR_RATING'

# Select relevant columns
data = data[input_columns + [output_column]]

# Handle missing values
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data = data.dropna()

# Convert to numeric
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data = data.dropna()

# Extract X and y
X = data[input_columns].values
y = data[output_column].values
y = to_categorical(y - 1)  # convert 1-5 stars to 0-4, then one-hot encode

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
with open('doctor_new_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape for CNN input (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build CNN model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(32, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # 5 star classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Save the CNN model
model.save('doctor_new_cnn.keras')

print("CNN model and scaler saved successfully.")'''


'\n# Load the dataset\ndata = pd.read_csv(\'doctor_new.csv\')\n\n# Define input and output columns\ninput_columns = [\n    \'H_COMP_2_A_P\', \'H_COMP_2_SN_P\', \'H_COMP_2_U_P\',\n    \'H_DOCTOR_EXPLAIN_A_P\', \'H_DOCTOR_EXPLAIN_SN_P\', \'H_DOCTOR_EXPLAIN_U_P\',\n    \'H_DOCTOR_LISTEN_A_P\', \'H_DOCTOR_LISTEN_SN_P\', \'H_DOCTOR_LISTEN_U_P\',\n    \'H_DOCTOR_RESPECT_A_P\', \'H_DOCTOR_RESPECT_SN_P\', \'H_DOCTOR_RESPECT_U_P\'\n]\noutput_column = \'H_COMP_2_STAR_RATING\'\n\n# Select relevant columns\ndata = data[input_columns + [output_column]]\n\n# Handle missing values\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata = data.dropna()\n\n# Convert to numeric\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata = data.dropna()\n\n# Extract X and y\nX = data[input_columns].values\ny = data[output_column].values\ny = to_categorical(y - 1)  #

In [ ]:
'''
# Load the dataset
data = pd.read_csv('help_new.csv')

# Define input and output columns
input_columns = [
    'H_BATH_HELP_A_P', 'H_BATH_HELP_SN_P', 'H_BATH_HELP_U_P',
    'H_CALL_BUTTON_A_P', 'H_CALL_BUTTON_SN_P', 'H_CALL_BUTTON_U_P',
    'H_COMP_3_A_P', 'H_COMP_3_SN_P', 'H_COMP_3_U_P'
]
output_column = 'H_COMP_3_STAR_RATING'

# Select and clean data
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data = data.dropna()

for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data = data.dropna()

# Extract X and y
X = data[input_columns].values
y = data[output_column].values
y = to_categorical(y - 1)  # 1-5 stars → 0-4 → one-hot

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
with open('help_new_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape for Conv1D: (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build CNN model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(32, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # 5 star classes
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Save model
model.save('help_new_cnn.keras')

print("CNN model and scaler saved successfully.")'''


Epoch 1/50


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.5000 - loss: 1.4200 - val_accuracy: 0.7805 - val_loss: 0.7514
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7702 - loss: 0.6797 - val_accuracy: 0.8537 - val_loss: 0.3786
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8423 - loss: 0.4038 - val_accuracy: 0.8963 - val_loss: 0.2545
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8817 - loss: 0.2944 - val_accuracy: 0.9126 - val_loss: 0.2395
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9076 - loss: 0.2435 - val_accuracy: 0.9207 - val_loss: 0.1899
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8856 - loss: 0.2714 - val_accuracy: 0.9289 - val_loss: 0.1734
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9108 - loss: 0.2344 - val_accuracy: 0.9329 - val_loss: 0.1690
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9051 - loss: 0.2188 - val_accuracy: 0.9451 - val_loss: 0.15

In [7]:
'''
# Load the dataset
data = pd.read_csv('left_hospital.csv')

# Define input and output columns
input_columns = ['H_DISCH_HELP_N_P', 'H_DISCH_HELP_Y_P', 'H_SYMPTOMS_N_P', 'H_SYMPTOMS_Y_P']
output_column = 'H_COMP_6_STAR_RATING'

# Preprocess data
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data = data.dropna()

# Convert to numeric
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data = data.dropna()

# Prepare features and labels
X = data[input_columns].values
y = to_categorical(data[output_column].values - 1)  # Convert to 0-based and one-hot

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
with open('left_hospital_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape for CNN input: (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build CNN model
model = Sequential([
    Conv1D(64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=1),
    Dropout(0.3),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # 5 star classes
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Save model
model.save('left_hospital_cnn.keras')

print("CNN model and scaler saved successfully.")'''


'\n# Load the dataset\ndata = pd.read_csv(\'left_hospital.csv\')\n\n# Define input and output columns\ninput_columns = [\'H_DISCH_HELP_N_P\', \'H_DISCH_HELP_Y_P\', \'H_SYMPTOMS_N_P\', \'H_SYMPTOMS_Y_P\']\noutput_column = \'H_COMP_6_STAR_RATING\'\n\n# Preprocess data\ndata = data[input_columns + [output_column]]\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata = data.dropna()\n\n# Convert to numeric\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata = data.dropna()\n\n# Prepare features and labels\nX = data[input_columns].values\ny = to_categorical(data[output_column].values - 1)  # Convert to 0-based and one-hot\n\n# Train/test split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Scale features\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_test = scaler.trans

In [8]:
'''# Load the dataset
data = pd.read_csv('nurse_new.csv')

# Define input and output columns
input_columns = [
    'H_COMP_1_A_P', 'H_COMP_1_SN_P', 'H_COMP_1_U_P',
    'H_NURSE_EXPLAIN_A_P', 'H_NURSE_EXPLAIN_SN_P', 'H_NURSE_EXPLAIN_U_P',
    'H_NURSE_LISTEN_A_P', 'H_NURSE_LISTEN_SN_P', 'H_NURSE_LISTEN_U_P',
    'H_NURSE_RESPECT_A_P', 'H_NURSE_RESPECT_SN_P', 'H_NURSE_RESPECT_U_P'
]
output_column = 'H_COMP_1_STAR_RATING'

# Select and clean the data
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data = data.dropna()
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data = data.dropna()

# Prepare input and output
X = data[input_columns].values
y = data[output_column].values
y = to_categorical(y - 1)  # One-hot encode (1–5 → 0–4)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale input
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
with open('nurse_new_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape for CNN input (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build CNN model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(32, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # 5 star classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Save model
model.save('nurse_new_cnn.keras')

print("✅ CNN model and scaler saved successfully.")'''


'# Load the dataset\ndata = pd.read_csv(\'nurse_new.csv\')\n\n# Define input and output columns\ninput_columns = [\n    \'H_COMP_1_A_P\', \'H_COMP_1_SN_P\', \'H_COMP_1_U_P\',\n    \'H_NURSE_EXPLAIN_A_P\', \'H_NURSE_EXPLAIN_SN_P\', \'H_NURSE_EXPLAIN_U_P\',\n    \'H_NURSE_LISTEN_A_P\', \'H_NURSE_LISTEN_SN_P\', \'H_NURSE_LISTEN_U_P\',\n    \'H_NURSE_RESPECT_A_P\', \'H_NURSE_RESPECT_SN_P\', \'H_NURSE_RESPECT_U_P\'\n]\noutput_column = \'H_COMP_1_STAR_RATING\'\n\n# Select and clean the data\ndata = data[input_columns + [output_column]]\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata = data.dropna()\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata = data.dropna()\n\n# Prepare input and output\nX = data[input_columns].values\ny = data[output_column].values\ny = to_categorical(y - 1)  # One-hot encode (1–5 → 0–4)\n\n# Train/test split\nX

In [9]:
'''
# Load the dataset
data = pd.read_csv('quite.csv')

# Define input and output columns
input_columns = ['H_QUIET_HSP_A_P', 'H_QUIET_HSP_SN_P', 'H_QUIET_HSP_U_P']
output_column = 'H_QUIET_STAR_RATING'

# Select relevant columns and clean
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data.dropna(inplace=True)

# Convert to numeric
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data.dropna(inplace=True)

# Extract features and labels
X = data[input_columns].values
y = to_categorical(data[output_column].values.astype(int) - 1)  # One-hot encoding (0–4)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale inputs
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
with open('quiet_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape input for Conv1D: (samples, timesteps, features)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')  # 5 classes for 1-5 star ratings
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train_cnn, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate
loss, acc = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

# Save model
model.save('quiet_cnn.keras')

print("✅ CNN model and scaler saved as 'quiet_cnn.keras' and 'quiet_scaler.pkl'")'''


'\n# Load the dataset\ndata = pd.read_csv(\'quite.csv\')\n\n# Define input and output columns\ninput_columns = [\'H_QUIET_HSP_A_P\', \'H_QUIET_HSP_SN_P\', \'H_QUIET_HSP_U_P\']\noutput_column = \'H_QUIET_STAR_RATING\'\n\n# Select relevant columns and clean\ndata = data[input_columns + [output_column]]\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata.dropna(inplace=True)\n\n# Convert to numeric\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata.dropna(inplace=True)\n\n# Extract features and labels\nX = data[input_columns].values\ny = to_categorical(data[output_column].values.astype(int) - 1)  # One-hot encoding (0–4)\n\n# Train-test split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Scale inputs\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_test = scaler.tran

In [11]:
'''
# Load the dataset
data = pd.read_csv('recommend.csv')

# Define input and output columns
input_columns = ['H_RECMND_DN', 'H_RECMND_DY', 'H_RECMND_PY']
output_column = 'H_COMP_1_STAR_RATING'

# Select relevant columns and clean
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data.dropna(inplace=True)

# Convert to numeric
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data.dropna(inplace=True)

# Extract features and labels
X = data[input_columns].values
y = to_categorical(data[output_column].astype(int) - 1, num_classes=5)  # 0–4 classes

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale inputs
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
with open('recommend_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape for Conv1D: (samples, timesteps, features)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')  # 5 output classes (1–5 star rating)
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_cnn, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate model
loss, acc = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

# Save the trained CNN model
model.save('recommend_cnn.keras')

print("✅ CNN model and scaler saved as 'recommend_cnn.keras' and 'recommend_scaler.pkl'")'''


'\n# Load the dataset\ndata = pd.read_csv(\'recommend.csv\')\n\n# Define input and output columns\ninput_columns = [\'H_RECMND_DN\', \'H_RECMND_DY\', \'H_RECMND_PY\']\noutput_column = \'H_COMP_1_STAR_RATING\'\n\n# Select relevant columns and clean\ndata = data[input_columns + [output_column]]\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata.dropna(inplace=True)\n\n# Convert to numeric\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata.dropna(inplace=True)\n\n# Extract features and labels\nX = data[input_columns].values\ny = to_categorical(data[output_column].astype(int) - 1, num_classes=5)  # 0–4 classes\n\n# Train-test split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Scale inputs\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_test = scaler.transform(X_tes

In [12]:
'''
# Load dataset
data = pd.read_csv('staff_new.csv')

# Define input and output columns
input_columns = [
    'H_COMP_5_A_P', 'H_COMP_5_SN_P', 'H_COMP_5_U_P',
    'H_MED_FOR_A_P', 'H_MED_FOR_SN_P', 'H_MED_FOR_U_P',
    'H_SIDE_EFFECTS_A_P', 'H_SIDE_EFFECTS_SN_P', 'H_SIDE_EFFECTS_U_P'
]
output_column = 'H_COMP_5_STAR_RATING'

# Clean data
data = data[input_columns + [output_column]]
data.replace(['Not Available', '', 'NA', 'N/A'], np.nan, inplace=True)
data.dropna(inplace=True)

# Convert all columns to numeric
for col in input_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[output_column] = pd.to_numeric(data[output_column], errors='coerce')
data.dropna(inplace=True)

# Prepare features and labels
X = data[input_columns].values
y = to_categorical(data[output_column].astype(int) - 1)  # 0-based labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save scaler
with open('staff_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape input for Conv1D (samples, timesteps, features)
X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Build CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')  # 5 classes (1-5 star ratings)
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train_cnn, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate
loss, acc = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

# Save model
model.save('staff_new_cnn.keras')
'''

'\n# Load dataset\ndata = pd.read_csv(\'staff_new.csv\')\n\n# Define input and output columns\ninput_columns = [\n    \'H_COMP_5_A_P\', \'H_COMP_5_SN_P\', \'H_COMP_5_U_P\',\n    \'H_MED_FOR_A_P\', \'H_MED_FOR_SN_P\', \'H_MED_FOR_U_P\',\n    \'H_SIDE_EFFECTS_A_P\', \'H_SIDE_EFFECTS_SN_P\', \'H_SIDE_EFFECTS_U_P\'\n]\noutput_column = \'H_COMP_5_STAR_RATING\'\n\n# Clean data\ndata = data[input_columns + [output_column]]\ndata.replace([\'Not Available\', \'\', \'NA\', \'N/A\'], np.nan, inplace=True)\ndata.dropna(inplace=True)\n\n# Convert all columns to numeric\nfor col in input_columns:\n    data[col] = pd.to_numeric(data[col], errors=\'coerce\')\ndata[output_column] = pd.to_numeric(data[output_column], errors=\'coerce\')\ndata.dropna(inplace=True)\n\n# Prepare features and labels\nX = data[input_columns].values\ny = to_categorical(data[output_column].astype(int) - 1)  # 0-based labels\n\n# Split data\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42

In [2]:


# Load the dataset
data = pd.read_csv('rate.csv')

# Replace 'Not Available' with NaN, then drop incomplete rows
data.replace('Not Available', np.nan, inplace=True)
data = data.dropna(subset=[
    'H_HSP_RATING_0_6',
    'H_HSP_RATING_7_8',
    'H_HSP_RATING_9_10',
    'H_HSP_RATING_STAR_RATING'
])

# Convert columns to numeric
for col in ['H_HSP_RATING_0_6', 'H_HSP_RATING_7_8', 'H_HSP_RATING_9_10', 'H_HSP_RATING_STAR_RATING']:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data = data.dropna()

# Prepare input features (X) and target (y)
X = data[['H_HSP_RATING_0_6', 'H_HSP_RATING_7_8', 'H_HSP_RATING_9_10']].values
y = data['H_HSP_RATING_STAR_RATING'].astype(int).values - 1  # zero-based (1→0, 5→4)
y = to_categorical(y, num_classes=5)                        # one-hot encode, 5 classes

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# Scale inputs
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler for future use
with open('star_rating_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Reshape for Conv1D: (samples, timesteps=3, channels=1)
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(3, 1)),
    Dropout(0.2),
    Conv1D(filters=32, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(5, activation='softmax')   # 5 star classes
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train_cnn, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.20,
    verbose=1
)

# Evaluate on test set
loss, accuracy = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Save the CNN model
model.save('hospital_star_rating_cnn.keras')

print("✅ CNN model and scaler saved successfully.")


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.4365 - loss: 1.4742 - val_accuracy: 0.6240 - val_loss: 1.0004
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6979 - loss: 0.9089 - val_accuracy: 0.8069 - val_loss: 0.5973
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8350 - loss: 0.5438 - val_accuracy: 0.8720 - val_loss: 0.4039
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8563 - loss: 0.4014 - val_accuracy: 0.8902 - val_loss: 0.3212
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8691 - loss: 0.3455 - val_accuracy: 0.8740 - val_loss: 0.3019
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8573 - loss: 0.3180 - val_accuracy: 0.8862 - val_loss: 0.2806
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8718 - loss: 0.3077 - val_accuracy: 0.8821 - val_loss: 0.2755
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8711 - loss: 0.2916 - val_accuracy: 0.8882 - val_loss